In [60]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Inicia el navegador (asegúrate de tener el controlador adecuado para tu navegador)
driver = webdriver.Chrome()  # O usa Firefox() si prefieres

# Navega a la web de JobLeads
driver.get("https://www.jobleads.com/es")

# Espera hasta que el icono del menú esté visible
menu_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//*[@aria-label="Menu desplegable"]'))  # Modifica el Xpath según el botón del menú
)

# Haz clic en el icono del menú desplegable
menu_button.click()

# Espera un momento para permitir que el menú se despliegue
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, 'c-header-alternative-mobile-body--active'))
)

# Verifica si el <body> tiene la clase que indica que el menú está abierto
body_class = driver.find_element(By.TAG_NAME, 'body').get_attribute('class')
if "c-header-alternative-mobile-body--active" in body_class:
    print("El menú desplegable se abrió correctamente.")
else:
    print("El menú no se abrió.")

# Aquí puedes proceder a automatizar el inicio de sesión u otras acciones


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=130.0.6723.70)
Stacktrace:
0   chromedriver                        0x00000001051f7634 cxxbridge1$str$ptr + 3645404
1   chromedriver                        0x00000001051efe94 cxxbridge1$str$ptr + 3614780
2   chromedriver                        0x0000000104c5c104 cxxbridge1$string$len + 88416
3   chromedriver                        0x0000000104c37b70 core::str::slice_error_fail::h1cab30ac4b13c655 + 3792
4   chromedriver                        0x0000000104cc466c cxxbridge1$string$len + 515784
5   chromedriver                        0x0000000104cd7638 cxxbridge1$string$len + 593556
6   chromedriver                        0x0000000104c92f54 cxxbridge1$string$len + 313264
7   chromedriver                        0x0000000104c93ba4 cxxbridge1$string$len + 316416
8   chromedriver                        0x00000001051c21d4 cxxbridge1$str$ptr + 3427196
9   chr

In [54]:
posiciones_data

[]

In [115]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from time import sleep
# Iniciamos el driver de Chrome para acceder a este
driver = webdriver.Chrome()

# Navegamos a la página web
driver.get("https://www.jobleads.com/es")

# Aceptamos el banner de coookies
accept_cookies_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Aceptar")]'))  # Modifica el XPATH según sea necesario
)

# Hacemos clic en el botón de "Aceptar cookies"
accept_cookies_button.click()

# Encontramos el botón de iniciar sesión y hacemos click
login_button = driver.find_element(By.XPATH, '//*[@id="__nuxt"]/div[1]/div[1]/div[1]/div/div/div[2]/button')
login_button.click()

# Esperamos hasta que el campo de correo electrónico esté presente
email_input = WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.XPATH, '//input[@type="email" and @data-testid="login-form-email-field-ui-input"]'))
)
email_input.send_keys("cakapa61@gmail.com")

password_input = WebDriverWait(driver, 3).until(
    EC.presence_of_element_located((By.XPATH, '//input[@type="password" and @data-testid="login-form-password-field-ui-input"]'))  # Cambia el data-testid si es necesario
)
# Ingresa la contraseña
password_input.send_keys("DaB01.proyecto")

# Si tienes un botón de inicio de sesión, puedes hacer clic en él
login_button = WebDriverWait(driver, 3).until(
    EC.element_to_be_clickable((By.XPATH, '/html/body/div[4]/div/div/form/div[3]/button'))
)
login_button.click()

# Ya hemos conseguido iniciar sesión para acceder a la información oculta, ahora intentemos extraer dicha información
# Para ello, debemos ir al área de "Trabajos" y desde ahí buscar el resto

try:
    trabajos_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//a[@data-testid="app-header-jobs-menu-item"]'))
    )
    
    # Hacemos clic en el botón "Trabajos"
    trabajos_button.click()
except StaleElementReferenceException:
    print("El elemento ya no está presente en el DOM. Intentando nuevamente...")
    
    # Reencontramos el botón y hacer clic de nuevo
    trabajos_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//a[@data-testid="app-header-jobs-menu-item"]'))
    )
    trabajos_button.click()

# Una vez estamos aquí, hemos de posicionarnos para buscar las posiciones deseadas, para ello debemos aplicar la búsqueda:
busqueda_element = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//div[@class="search-switch__item flex gap-1 items-center search-switch__item_active"]'))
)
busqueda_element.click()

#############################

# Lista de posiciones a buscar
posiciones = ["Analista de datos", "Controller", "Data Analyst", "Analista Power BI", "Data Scientist", "Data Science","Web Developer"]
posiciones_data = []

# Iteramos sobre la lista de posiciones
for posicion in posiciones:
    # Esperamos a que el campo de búsqueda esté presente
    search_input = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.XPATH, '//input[@data-testid="search-form-keyword-ui-input"]'))
    )

    # Limpiar el campo de búsqueda para la siguiente posición
    search_input.clear()

    # Enviamos la posición al campo de búsqueda
    search_input.send_keys(posicion)

    # Esperamos hasta que el botón de búsqueda esté clickable
    search_button = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="ui-button"]'))
    )

    # Hacemos clic en el botón de búsqueda
    search_button.click()

    try:
        close_popup = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, '//span[@class="jlu-icon jlu-icon_close shop-overlay__close"]'))
        )
        close_popup.click()
    except TimeoutException:
        print("No hay pop-up para cerrar.")

    # Ahora extraemos los datos de la lista de trabajos
    job_list = WebDriverWait(driver, 15).until(
        EC.presence_of_all_elements_located((By.XPATH, '//div[@class="serp-list js-serp-list"]/li'))
    )

    for job in job_list:
        title = job.find_element(By.TAG_NAME, 'h4').text if job.find_element(By.TAG_NAME, 'h4') else 'Sin título'
        
        company = job.find_element(By.CLASS_NAME, 'company').text if job.find_element(By.CLASS_NAME, 'company') else 'Sin empresa'
        location = job.find_element(By.CLASS_NAME, 'company-address').text if job.find_element(By.CLASS_NAME, 'company-address') else 'Sin ubicación'
        description = job.find_element(By.CLASS_NAME, 'description').text if job.find_element(By.CLASS_NAME, 'description') else 'Sin descripción'

        # Añadir el trabajo a la lista
        posiciones_data.append({
            'Título': title,
            'Empresa': company,
            'Ubicación': location,
            'Descripción': description,
            'Posición': posicion
        })

    # Esperamos unos segundos para que los resultados se carguen
    sleep(2)

El elemento ya no está presente en el DOM. Intentando nuevamente...


KeyboardInterrupt: 

In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
import requests


# Inicializar driver de Chrome
driver = webdriver.Chrome()

# Navegamos a la página web
driver.get("https://www.jobleads.com/es")

# Aceptamos el banner de coookies
accept_cookies_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Aceptar")]'))  # Modifica el XPATH según sea necesario
)

# Hacemos clic en el botón de "Aceptar cookies"
accept_cookies_button.click()

# Encontramos el botón de iniciar sesión y hacemos click
login_button = driver.find_element(By.XPATH, '//*[@id="__nuxt"]/div[1]/div[1]/div[1]/div/div/div[2]/button')
login_button.click()

# Esperamos hasta que el campo de correo electrónico esté presente
email_input = WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.XPATH, '//input[@type="email" and @data-testid="login-form-email-field-ui-input"]'))
)
email_input.send_keys("cakapa61@gmail.com")

password_input = WebDriverWait(driver, 3).until(
    EC.presence_of_element_located((By.XPATH, '//input[@type="password" and @data-testid="login-form-password-field-ui-input"]'))  # Cambia el data-testid si es necesario
)
# Ingresa la contraseña
password_input.send_keys("DaB01.proyecto")

# Si tienes un botón de inicio de sesión, puedes hacer clic en él
login_button = WebDriverWait(driver, 3).until(
    EC.element_to_be_clickable((By.XPATH, '/html/body/div[4]/div/div/form/div[3]/button'))
)
login_button.click()

# Ya hemos conseguido iniciar sesión para acceder a la información oculta, ahora intentemos extraer dicha información
# Para ello, debemos ir al área de "Trabajos" y desde ahí buscar el resto

try:
    trabajos_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//a[@data-testid="app-header-jobs-menu-item"]'))
    )
    
    # Hacemos clic en el botón "Trabajos"
    trabajos_button.click()
except StaleElementReferenceException:
    print("El elemento ya no está presente en el DOM. Intentando nuevamente...")
    
    # Reencontramos el botón y hacer clic de nuevo
    trabajos_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//a[@data-testid="app-header-jobs-menu-item"]'))
    )
    trabajos_button.click()
    pass

# Una vez estamos aquí, hemos de posicionarnos para buscar las posiciones deseadas, para ello debemos aplicar la búsqueda:
busqueda_element = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//div[@class="search-switch__item flex gap-1 items-center search-switch__item_active"]'))
)
busqueda_element.click()

#############################

# Definir las palabras clave y ubicaciones
keywords = ["Analista de datos", "Data Scientist", "Data Analyst"]
locations = ["Madrid", "Barcelona"]

# Lista para almacenar las URLs de trabajos
job_urls = []

# Iterar sobre las combinaciones de keywords y locations
for keyword in keywords:
    for location in locations:
        # Construir la URL de búsqueda con los parámetros deseados
        url = f"https://www.jobleads.com/search/jobs?keywords={keyword}&location={location}&location_country=ESP"
        
        # Hacer la solicitud a la URL
        response = requests.get(url)
        
        # Verificar que la solicitud fue exitosa
        if response.status_code == 200:
            print(f"La llamada a {response.url} fue exitosa.")
            
            # Analizar el HTML de la página de resultados
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Extraer y añadir cada URL de oferta a la lista principal
            for job in soup.find_all("a", class_="absolute left-0 right-0 top-0 bottom-0 opacity-0 z-[1] cursor-pointer"):
                job_url = f"https://jobleads.com{job['href']}"
                job_urls.append(job_url)
            
        else:
            print(f"Error en la solicitud a {url}: Status code {response.status_code}")

# Imprimir las URLs obtenidas para verificar
print(job_urls)

#Para extraer los distintos enlaces a las ofertas, ponemos el siguiente código

soup1 = BeautifulSoup(driver.page_source, "html.parser")
[f"https://jobleads.com{x["href"]}" for x in soup1.find_all("a", class_ = "absolute left-0 right-0 top-0 bottom-0 opacity-0 z-[1] cursor-pointer")]

#
#    try:
#        close_popup = WebDriverWait(driver, 5).until(
#            EC.element_to_be_clickable((By.XPATH, '//span[@class="jlu-icon jlu-icon_close shop-overlay__close"]'))
#        )
#        close_popup.click()
#    except TimeoutException:
#        print("No hay pop-up para cerrar.")
#
#    sleep(5)
#    # Ahora extraemos los datos de la lista de trabajos
#    lista_trabajos = WebDriverWait(driver, 20).until(
#        EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#__nuxt > div > div.app-main > div > div > div.mt-4.flex.flex-col.items-center.gap-6.lg\:items-start.lg\:flex-row'))
#    )
#
#soup1 = BeautifulSoup(driver.page_source, "html.parser")
#[f"https://jobleads.com{x["href"]}" for x in soup1.find_all("a", class_ = "absolute left-0 right-0 top-0 bottom-0 opacity-0 z-[1] cursor-pointer")]
#
#
#
#    # Esperamos unos segundos para que los resultados se carguen
#    sleep(2)
#    # Obtener la fuente de la página cargada y extraer datos con BeautifulSoup
#    page_source = driver.page_source
#    trabajos = extraer_trabajos(page_source)
#    posiciones_data.extend(trabajos)
#
#    # Espera opcional
#    sleep(2)
#
## Convertir datos a DataFrame para análisis
#df = pd.DataFrame(posiciones_data)
#print(df)
#
## Cerrar el navegador
#driver.quit()


El elemento ya no está presente en el DOM. Intentando nuevamente...
La llamada a https://www.jobleads.com/search/jobs?keywords=Analista%20de%20datos&location=Madrid&location_country=ESP fue exitosa.
La llamada a https://www.jobleads.com/search/jobs?keywords=Analista%20de%20datos&location=Barcelona&location_country=ESP fue exitosa.
La llamada a https://www.jobleads.com/search/jobs?keywords=Data%20Scientist&location=Madrid&location_country=ESP fue exitosa.
La llamada a https://www.jobleads.com/search/jobs?keywords=Data%20Scientist&location=Barcelona&location_country=ESP fue exitosa.
La llamada a https://www.jobleads.com/search/jobs?keywords=Data%20Analyst&location=Madrid&location_country=ESP fue exitosa.
La llamada a https://www.jobleads.com/search/jobs?keywords=Data%20Analyst&location=Barcelona&location_country=ESP fue exitosa.
[]


['https://jobleads.com/job/ec8650841727a183f700b02b2b8b60ad0',
 'https://jobleads.com/job/e5136e097b065bc9459fbbbb677978621',
 'https://jobleads.com/job/e69787bb7d74e95894afc379b66ca877a',
 'https://jobleads.com/job/edbbadd5b4dcc5da6166bae014c9e45d9',
 'https://jobleads.com/job/e99fd70c1e9ac5076e9c98d6b3c3ce328',
 'https://jobleads.com/job/ec6a822d5517948b2e0e6799bed0c1b0c',
 'https://jobleads.com/job/e50eefeca633479dcef4ffb634359aa1c',
 'https://jobleads.com/job/e523188ad402661960006fcc6ac0e6ee1',
 'https://jobleads.com/job/ec400cf03905b234766297f8502179588',
 'https://jobleads.com/job/eeddaf96ad5496fee30309db23c606bb3',
 'https://jobleads.com/job/e5565efacdf0af7d2b7905de30faabfb3',
 'https://jobleads.com/job/e9c28db2ca4b8a78d4a8af56c864ea16b',
 'https://jobleads.com/job/eba5dfbd8217b76b7ee4dffa0e3d08c18',
 'https://jobleads.com/job/e2261fec442ef44dc196eba27c528b97f',
 'https://jobleads.com/job/ebe4777481c0726ee2b40f0585de382ca',
 'https://jobleads.com/job/e103e1643331d0b03fb7ba83d368

In [116]:
soup1 = BeautifulSoup(driver.page_source, "html.parser")
[f"https://jobleads.com{x["href"]}" for x in soup1.find_all("a", class_ = "absolute left-0 right-0 top-0 bottom-0 opacity-0 z-[1] cursor-pointer")]

['https://jobleads.com/job/eba2bf38bdc4c66ba2f5f90be22d72dd5',
 'https://jobleads.com/job/ee3e4547131536d8a8d578b60a84a03b6',
 'https://jobleads.com/job/e9816b3dfbcdf164b845b3957c611dc8a',
 'https://jobleads.com/job/e4f959c7e599498871442801725cab32e',
 'https://jobleads.com/job/e28ec128a2c1ae1e72cb35f99e26e6ba8',
 'https://jobleads.com/job/e15686a17f923f04e420b770a7e107172',
 'https://jobleads.com/job/e35b1fba3ad91791cd1e7eef2e1316bd5',
 'https://jobleads.com/job/e72734c49dd0e32517c777231db345015',
 'https://jobleads.com/job/ec8e1cf7ec834ba7ce04c05eab1dfe585',
 'https://jobleads.com/job/e9285b1db6ec30638083df67df0b1f14c',
 'https://jobleads.com/job/e988d48d61765e230be6f9c057ddd6d08',
 'https://jobleads.com/job/e29efbdfb6417e2741fb231cfa3f9b560',
 'https://jobleads.com/job/e1659c4499c752859577d7b1ca62e0432',
 'https://jobleads.com/job/e83dbbba17ce847ac17cba9d3c0b2c43c',
 'https://jobleads.com/job/e61d5890d2de4221d7a2699d5b016b477',
 'https://jobleads.com/job/e63cedaaf1ec3c8d8f99cb5f2ce9

In [ ]:
# Función para extraer las URLs de una página de resultados
def extract_job_urls(soup):
    job_urls = []
    for job in soup.find_all("a", class_="absolute left-0 right-0 top-0 bottom-0 opacity-0 z-[1] cursor-pointer"):
        job_url = f"https://jobleads.com{job['href']}"
        job_urls.append(job_url)
    return job_urls

# Bucle para manejar la paginación (si es necesario)
while True:
    # ... (código para realizar la búsqueda)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    job_urls.extend(extract_job_urls(soup))

    # Buscar el elemento de paginación (si existe)
    next_page_button = driver.find_element(By.XPATH, '//button[text()="Siguiente"]')  # Ajusta el XPath
    if not next_page_button.is_enabled():
        break

    next_page_button.click()

In [28]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
from time import sleep
import requests


# Inicializar driver de Chrome
driver = webdriver.Chrome()

# Navegamos a la página web
driver.get("https://www.jobleads.com/es")

# Aceptamos el banner de coookies
accept_cookies_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Aceptar")]'))  # Modifica el XPATH según sea necesario
)

# Hacemos clic en el botón de "Aceptar cookies"
accept_cookies_button.click()

# Encontramos el botón de iniciar sesión y hacemos click
login_button = driver.find_element(By.XPATH, '//*[@id="__nuxt"]/div[1]/div[1]/div[1]/div/div/div[2]/button')
login_button.click()

# Esperamos hasta que el campo de correo electrónico esté presente
email_input = WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.XPATH, '//input[@type="email" and @data-testid="login-form-email-field-ui-input"]'))
)
email_input.send_keys("cakapa61@gmail.com")

password_input = WebDriverWait(driver, 3).until(
    EC.presence_of_element_located((By.XPATH, '//input[@type="password" and @data-testid="login-form-password-field-ui-input"]'))  # Cambia el data-testid si es necesario
)
# Ingresa la contraseña
password_input.send_keys("DaB01.proyecto")

# Si tienes un botón de inicio de sesión, puedes hacer clic en él
login_button = WebDriverWait(driver, 3).until(
    EC.element_to_be_clickable((By.XPATH, '/html/body/div[4]/div/div/form/div[3]/button'))
)
login_button.click()

# Ya hemos conseguido iniciar sesión para acceder a la información oculta, ahora intentemos extraer dicha información
# Para ello, debemos ir al área de "Trabajos" y desde ahí buscar el resto

try:
    trabajos_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//a[@data-testid="app-header-jobs-menu-item"]'))
    )
    
    # Hacemos clic en el botón "Trabajos"
    trabajos_button.click()
except StaleElementReferenceException:
    
    # Reencontramos el botón y hacer clic de nuevo
    trabajos_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//a[@data-testid="app-header-jobs-menu-item"]'))
    )
    trabajos_button.click()
    pass

# Definir las palabras clave y ubicaciones
keywords = ["Analyst", "Data Scientist", "Data Engineer"]
locations = ["Madrid", "Barcelona"]

# Lista para almacenar las URLs de trabajos
job_urls = []

# Iterar sobre las combinaciones de keywords y locations
for keyword in keywords:
    for location in locations:
        for num in range(1,8):
        # Construir la URL de búsqueda con los parámetros deseados
            url = f"https://www.jobleads.com/search/jobs?keywords={keyword}&location={location}&location_country=ESP&page={num}"

            # Hacer la solicitud a la URL
            driver.get(url)

            # Esperar a que la página se cargue y extraer URLs
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'h2')))

            soup1 = BeautifulSoup(driver.page_source, "html.parser")

            # Extraer las URLs de cada oferta
            job_urls.extend(
                [f"https://jobleads.com{x['href']}" for x in soup1.find_all("a", class_="absolute left-0 right-0 top-0 bottom-0 opacity-0 z-[1] cursor-pointer")]
            )

# Lista para almacenar los datos de posiciones
posiciones_data = []

# Extraer información de cada URL
for url in job_urls:
    driver.get(url)
    
    # Esperar a que la página se cargue
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'h2')))
    
    soup1 = BeautifulSoup(driver.page_source, "html.parser")
    
    # Extraemos datos
    title = soup1.find('h2', class_='text-xl font-bold text-black').text if soup1.find('h2', class_='text-xl font-bold text-black') else 'Sin título'
    
    location = soup1.find('div', class_='py-1 rounded-full bg-[#f4f4f4] text-tp px-3 text-sm').text if soup1.find('div', class_='py-1 rounded-full bg-[#f4f4f4] text-tp px-3 text-sm') else 'Sin ubicación'
    
    description = soup1.find('p', class_='mt-3 text-base text-[#262D37]').text if soup1.find('p', class_='mt-3 text-base text-[#262D37]') else 'Sin descripción'
    
    servicios = [servicio.text for servicio in soup1.find_all('div', class_='flex gap-2 flex-wrap mt-3')] if soup1.find_all('div', class_='flex gap-2 flex-wrap mt-3') else []

    timestamp_extraccion = pd.Timestamp.now()
    # En el caso del sueldo, encontramos una complicación ya que aporta un rango y no siempre se encuentra en la misma posición, por lo que tenemos que realizar un pequeño apaño antes de descargar
    # En una primera instancia, ponemos como predefinido que no aparezca el salario
    sueldo = 'Sin sueldo'
    # Ahora que tenemos algo por defecto, intentemos sortear el código
    divs = soup1.find_all('div', class_='py-1 rounded-full bg-[#f4f4f4] text-tp px-3 text-sm')
    for div in divs:
        if "EUR" in div.text:
            sueldo = div.text
    # Añadir la posición a la lista
    posiciones_data.append({
        'Título': title,
        'Ubicación': location,
        'Descripción': description,
        'Servicios': servicios,
        'Sueldo':sueldo,
        'Timestamp': timestamp_extraccion,
        'Url':url
    })


# Cerramos el controlador
driver.quit()

# Ahora que tenemos datos, tenemos que crear un df

df = pd.DataFrame(posiciones_data)

print(df)

df.to_csv('portales_empleo.csv', index=True)


                                             Título  Ubicación  \
0                                      Data Analyst     Madrid   
1       Business Intelligence Analyst (all genders)     Madrid   
2        Research Analyst (6 month Maternity Cover)     Madrid   
3         Senior Program Manager, Analyst Relations     Madrid   
4                            [RJ072] | Data Analyst     Madrid   
...                                             ...        ...   
1038                            Senior Data Analyst  Barcelona   
1039  Data Scientist Barcelona (Spain) Sep 27, 2024  Barcelona   
1040                                   Data Analyst  Barcelona   
1041    Jr Finance Data Analyst- Based in Barcelona  Barcelona   
1042                    Machine Learning Engineer 4  Barcelona   

                                            Descripción  \
0     Una empresa internacional y dinámica busca un ...   
1     An innovative edtech company is seeking a Busi...   
2     An established industry 

In [7]:
# ahora que tenemos datos, tenemos que crear un df

df = pd.DataFrame(posiciones_data)

In [21]:
posiciones_data

[{'Título': 'Data Analyst',
  'Ubicación': 'Madrid',
  'Descripción': 'Una empresa internacional y dinámica busca un Analista de Datos apasionado por la analítica y la colaboración. En este emocionante rol, serás parte de un equipo que apoya a las áreas comerciales mediante análisis descriptivos y recomendaciones basadas en datos. Te conectarás con diferentes partes interesadas para entender sus necesidades y generar análisis significativos que impulsen mejores decisiones. Con un enfoque en la flexibilidad y el bienestar del empleado, esta organización ofrece un ambiente de trabajo colaborativo donde tu crecimiento profesional es una prioridad. Si te encanta trabajar en equipo y disfrutar mientras lo haces, ¡este es tu lugar!',
  'Servicios': ['Tarjeta de restauranteSeguro de saludSeguro de vidaRemuneración flexibleTransporte gratuitoBeneficios bancariosCuidado 360º del empleado',
   ' Excel SAS SQL R Python Análisis de datos Comunicación',
   ' Licenciatura en matemáticas Licenciatura

In [20]:
df

0       https://jobleads.com/job/eba2bf38bdc4c66ba2f5f...
1       https://jobleads.com/job/e4b1388ab13374dab4103...
2       https://jobleads.com/job/ec2be137e14f9dbc1a8eb...
3       https://jobleads.com/job/ea5922eccaee3331c3aba...
4       https://jobleads.com/job/ea6ecb7b732973b55d2da...
                              ...                        
1017    https://jobleads.com/job/e09a569bd0aaa37bf0411...
1018    https://jobleads.com/job/e6c30361e7618acf8e876...
1019    https://jobleads.com/job/e095a7fdb6c3866944e2c...
1034    https://jobleads.com/job/ee55bb8c0f23269699759...
1035    https://jobleads.com/job/e4ac3a99eaaa5eaa22c82...
Name: Url, Length: 780, dtype: object